# **Distribución % de créditos por segmento (trimestre)**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "tulancingo" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_tulancingo',
 'df_financiamientos_2020_tulancingo',
 'df_financiamientos_2021_tulancingo',
 'df_financiamientos_2022_tulancingo',
 'df_financiamientos_2023_tulancingo',
 'df_financiamientos_2024_tulancingo']

In [2]:
df_financiamientos_2022_tulancingo = df_financiamientos_2022_tulancingo[['id','mes','monto']]
df_financiamientos_2023_tulancingo = df_financiamientos_2023_tulancingo[['id','mes','monto']]
df_financiamientos_2024_pachuca= df_financiamientos_2024_tulancingo[['id','mes','monto']]

## 2022

In [3]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2022_tulancingo['trimestre'] = df_financiamientos_2022_tulancingo['mes'].apply(asignar_trimestre)
df_financiamientos_2022_tulancingo

,id,mes,monto,trimestre
0,1,1,0.00,1
1,2,1,0.00,1
2,3,1,14398.00,1
3,4,1,96283.00,1
4,5,1,159720.57,1
...,...,...,...,...
311,312,12,2279250.00,4
312,313,12,2285795.72,4
313,314,12,2661350.00,4
314,315,12,2946484.00,4


In [4]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}
# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2022_tulancingo['segmento'] = df_financiamientos_2022_tulancingo['monto'].apply(asignar_segmento)
df_financiamientos_2022_tulancingo.head()

,id,mes,monto,trimestre,segmento
0,1,1,0.00,1,E
1,2,1,0.00,1,E
2,3,1,14398.00,1,E
3,4,1,96283.00,1,E
4,5,1,159720.57,1,E


In [5]:
df_financiamientos_2022_tulancingo = df_financiamientos_2022_tulancingo[['trimestre','segmento']]
df_financiamientos_2022_tulancingo

,trimestre,segmento
0,1,E
1,1,E
2,1,E
3,1,E
4,1,E
...,...,...
311,4,C
312,4,C
313,4,B
314,4,B


In [6]:
# Calcular el total de registros
total_registros = len(df_financiamientos_2022_tulancingo)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(df_financiamientos_2022_tulancingo, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  316
trimestre          1          2          3          4
segmento                                             
A           0.316456   0.000000   0.316456   0.000000
B           0.632911   0.949367   0.316456   0.632911
C           1.265823   1.265823   0.949367   2.531646
D           4.430380   5.379747   7.278481   4.430380
E          18.037975  12.658228  19.936709  18.037975
S           0.000000   0.316456   0.000000   0.316456


In [8]:
# Calcular el total de registros
total_registros = len(df_financiamientos_2022_tulancingo)
print("Total de registros: ", total_registros)

# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(df_financiamientos_2022_tulancingo, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

# Definir colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Crear la figura de la gráfica
fig = go.Figure()
trimestres = tabla_porcentajes.columns

# Reordenar la tabla de porcentajes según el orden de segmentos
tabla_porcentajes = tabla_porcentajes.reindex(orden_segmentos)

# Agregar trazos para cada trimestre
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))

# Actualizar diseño de la gráfica
fig.update_layout(
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y         
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title="Trimestre",
    barmode='group',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
    xaxis=dict(
        title='Segmento',  # Título del eje x
    ),
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)


guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_percent_2022', carpeta='assets/graficas')

# Mostrar la gráfica
fig.show()


Total de registros:  316
trimestre          1          2          3          4
segmento                                             
A           0.316456   0.000000   0.316456   0.000000
B           0.632911   0.949367   0.316456   0.632911
C           1.265823   1.265823   0.949367   2.531646
D           4.430380   5.379747   7.278481   4.430380
E          18.037975  12.658228  19.936709  18.037975
S           0.000000   0.316456   0.000000   0.316456


## 2023

In [9]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2023_tulancingo['trimestre'] = df_financiamientos_2023_tulancingo['mes'].apply(asignar_trimestre)
df_financiamientos_2023_tulancingo

,id,mes,monto,trimestre
0,1,1,1585485.28,1
1,2,1,608908.05,1
2,3,1,1918694.86,1
3,4,1,0.00,1
4,5,1,0.00,1
...,...,...,...,...
307,308,12,549386.31,4
308,309,12,585743.45,4
309,310,12,1383509.89,4
310,311,12,1316116.99,4


In [10]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2023_tulancingo['segmento'] = df_financiamientos_2023_tulancingo['monto'].apply(asignar_segmento)
df_financiamientos_2023_tulancingo.head()

,id,mes,monto,trimestre,segmento
0,1,1,1585485.28,1,D
1,2,1,608908.05,1,E
2,3,1,1918694.86,1,C
3,4,1,0.00,1,E
4,5,1,0.00,1,E


In [11]:
df_financiamientos_2023_tulancingo = df_financiamientos_2023_tulancingo[['trimestre','segmento']]
df_financiamientos_2023_tulancingo

,trimestre,segmento
0,1,D
1,1,E
2,1,C
3,1,E
4,1,E
...,...,...
307,4,E
308,4,E
309,4,D
310,4,D


In [12]:
# Calcular el total de registros
total_registros = len(df_financiamientos_2023_tulancingo)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(df_financiamientos_2023_tulancingo, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  312
trimestre          1          2          3          4
segmento                                             
A           0.000000   0.320513   0.000000   0.320513
B           0.641026   0.000000   0.000000   0.641026
C           2.564103   0.961538   1.923077   1.923077
D           4.166667   6.410256   6.410256   7.692308
E          13.461538  17.628205  16.346154  17.628205
S           0.320513   0.000000   0.320513   0.320513


In [13]:


# Definir colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Crear la figura de la gráfica
fig = go.Figure()
trimestres = tabla_porcentajes.columns
# Reordenar la tabla de porcentajes según el orden de segmentos
tabla_porcentajes = tabla_porcentajes.reindex(orden_segmentos)
# Agregar trazos para cada trimestre
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))

# Actualizar diseño de la gráfica
fig.update_layout(
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y         
    ),  
    xaxis=dict(
        title='Segmentos',  # Título del eje x
    ), 
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title="Trimestre",
    barmode='group',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_percent_2023', carpeta='assets/graficas')

# Mostrar la gráfica
fig.show()


## 2024

In [14]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2024_pachuca['trimestre'] = df_financiamientos_2024_pachuca['mes'].apply(asignar_trimestre)
df_financiamientos_2024_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3460\1484986024.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre
0,1,1,479825.86,1
1,2,1,527818.15,1
2,3,1,47387.00,1
3,4,1,0.00,1
4,5,1,0.00,1
...,...,...,...,...
192,193,7,769789.93,3
193,194,7,1308000.00,3
194,195,7,646000.00,3
195,196,7,667454.88,3


In [15]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}
# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2024_pachuca['segmento'] = df_financiamientos_2024_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2024_pachuca.head()

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3460\1690537017.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre,segmento
0,1,1,479825.86,1,E
1,2,1,527818.15,1,E
2,3,1,47387.00,1,E
3,4,1,0.00,1,E
4,5,1,0.00,1,E


In [16]:
df_financiamientos_2024_pachuca = df_financiamientos_2024_pachuca[['trimestre','segmento']]
df_financiamientos_2024_pachuca = df_financiamientos_2024_pachuca[df_financiamientos_2024_pachuca['trimestre']!=3]

df_financiamientos_2024_pachuca

,trimestre,segmento
0,1,E
1,1,E
2,1,E
3,1,E
4,1,E
...,...,...
154,2,E
155,2,E
156,2,E
157,2,E


In [17]:
# Calcular el total de registros
total_registros = len(df_financiamientos_2024_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(df_financiamientos_2024_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  159
trimestre          1          2
segmento                       
A           0.628931   0.000000
B           0.628931   3.773585
C           1.886792   1.257862
D           5.031447  11.949686
E          32.704403  40.880503
S           0.628931   0.628931


In [18]:


# Definir colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
orden_segmentos=["E", "D", "C", "B","A", "S", "L", 'ELITE']

# Crear la figura de la gráfica
fig = go.Figure()
trimestres = tabla_porcentajes.columns
# Reordenar la tabla de porcentajes según el orden de segmentos
tabla_porcentajes = tabla_porcentajes.reindex(orden_segmentos)

# Agregar trazos para cada trimestre
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))

# Actualizar diseño de la gráfica
fig.update_layout(
    yaxis=dict(
        title='%',  # Título del eje y
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y         
    ),  
    xaxis=dict(
        title='Segmentos',  # Título del eje y
    ),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title="Trimestre",
    barmode='group',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_percent_2024', carpeta='assets/graficas')

# Mostrar la gráfica
fig.show()
